In [ ]:
#import the libraries
import torch
import pandas as pd
import torch.nn as nn
import torch.optim as optim

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")

In [ ]:
#load the dataset
dataset = pd.read_csv("output.csv") #edit path if needed
dataset=dataset.drop(dataset.columns[[0]], axis=1)
X = dataset.iloc[:, 0:5].values
Y = dataset.iloc[:, 5:].values

In [ ]:
#split the dataset into training and testing
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0)

In [ ]:
#convert the data to torch tensors
x_train, y_train, x_test, y_test =torch.tensor(x_train, dtype=torch.float32, device=device), torch.tensor(y_train, dtype=torch.float32, device=device), torch.tensor(x_test, dtype=torch.float32, device=device),torch.tensor(y_test, dtype=torch.float32, device=device)

In [ ]:
#hyperparameters
#hyperparameters provided have been tuned using optuna
n_epoch = 30000 #number of epochs
batch_size = 80000 #batch size
lr = 0.006169410626430787 # learning rate
l1 = 127 # size of hidden layer 1

In [ ]:
#build the neural network
class PolarBear(nn.Module):
    def __init__(self,l1):
        super().__init__()
        self.input = nn.Linear(5,l1)
        self.input_act = nn.ReLU()
        self.output = nn.Linear(l1,3)
    def forward(self,x):
        x = self.input_act(self.input(x))
        x = self.output(x)
        return x 

In [ ]:
#training loop
def train(model, x_train, y_train, n_epoch,lr,batch_size, verbose=1):
    losses = []
    loss_fn = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    for epoch in range(n_epoch):
        for i in range(0,len(x_train),batch_size):
            Xbatch = x_train[i:i+batch_size]
            Ybatch = y_train[i:i+batch_size]
            y_pred = model(Xbatch)
            loss = loss_fn(y_pred, Ybatch)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        losses.append(float(loss))
        if verbose == 1:
            print(f'epoch: {epoch}/{n_epoch}, loss: {loss:.2f}', end='\r')
    return model, losses

In [ ]:
#train the model
model = PolarBear(l1)
model.to(device)
model, losses = train(model,x_train,y_train,n_epoch,lr,batch_size)

In [ ]:
#testing
r_count = 0
l_count = 0
for c in range(len(x_test)):
    result = torch.eq(torch.round(model(x_test[c])),y_test[c])
    for i in result:
        l = bool(i.item())
        if l == True:
            r_count += 1
        else: 
            l_count += 1
print(f'{r_count}/60000, accuracy = {r_count/60000}')

In [ ]:
#plot the training loss
%matplotlib inline
import matplotlib.pyplot as plt
y_axis = list(range(0, len(losses)))
plt.xlabel("epoch")
plt.ylabel("loss")
plt.title("Training")
plt.plot(y_axis,losses);

In [ ]:
#saving the model
torch.save(model.state_dict(), "model.pt")

In [ ]:
#loading the model
model.load_state_dict(torch.load("model.pt"))